In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)
from scipy.stats import mode

/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df_hist_trans = pd.read_csv('../input/historical_transactions.csv')
df_new_merchant_trans = pd.read_csv('../input/new_merchant_transactions.csv')

In [3]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)


In [4]:
df_hist_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [5]:
city_spends = df_hist_trans.groupby("city_id")["purchase_amount"].sum().to_frame()
city_spends.reset_index()

,city_id,purchase_amount
0,-1,-890862.015324
1,1,6877.017092
2,2,-8798.655021
3,3,-5546.779656
4,4,13480.692589
5,6,-14281.570867
6,7,-9413.597630
7,8,-53103.700252
8,9,-11778.094709
9,10,-10551.095902


In [6]:
city_spends.sort_values("purchase_amount").head(350)

,purchase_amount
city_id,
-1,-8.908620e+05
19,-2.656363e+05
88,-2.553767e+05
17,-2.225050e+05
331,-1.892611e+05
212,-1.454696e+05
53,-1.258602e+05
233,-1.204526e+05
57,-1.124054e+05


In [7]:
merchants = pd.read_csv("../input/merchants.csv")

In [8]:
merchants.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [9]:
df_train.corr()

,feature_1,feature_2,feature_3,target
feature_1,1.000000,-0.130969,0.583092,-0.014251
feature_2,-0.130969,1.000000,0.060925,-0.006242
feature_3,0.583092,0.060925,1.000000,-0.008125
target,-0.014251,-0.006242,-0.008125,1.000000


In [10]:
agg = {
    "numerical_1":['sum','max','min','mean','var'],
    "numerical_2":['sum','max','min','mean','var'],
    "category_1":{"mode" : lambda x : pd.Series.mode(x)[0]},
    "most_recent_sales_range":{"mode" : lambda x : pd.Series.mode(x)[0]},
    "most_recent_purchases_range":{"mode" : lambda x : pd.Series.mode(x)[0]},
    "avg_sales_lag3":['sum','max','min','mean','var'],
    "avg_purchases_lag3":['sum','max','min','mean','var'],
    "active_months_lag3":['sum','max','min','mean','var'],
    "avg_sales_lag6":['sum','max','min','mean','var'],
    "avg_purchases_lag6":['sum','max','min','mean','var'],
    "active_months_lag6":['sum','max','min','mean','var'],
    "avg_sales_lag12":['sum','max','min','mean','var'],    
    "avg_purchases_lag12":['sum','max','min','mean','var'],    
    "active_months_lag12":['sum','max','min','mean','var'],
    "category_4":{"mode" : lambda x : pd.Series.mode(x)},
    "category_2":{"mode" : lambda x : pd.Series.mode(x)},
    }

In [11]:
# merchant_aggregates = {}
# for col in ["merchant_group_id","merchant_category_id","subsector_id"] :
#     merchant_aggregates[col] = merchants.groupby(col).agg(agg)
#     merchant_aggregates[col].columns = ['_'.join(col).strip() for col in merchant_aggregates[col].columns.values]
# #     merchant_aggregates[col].reset_index(inplace=True)

In [12]:
len(df_hist_trans.columns.tolist())

14

In [13]:
# from pandas import Series
# merchant_to_merchant_group_id = Series(merchants.merchant_group_id.values,index=merchants.merchant_id).to_dict()

In [14]:
# df_hist_trans["merchant_group_id"] = df_hist_trans["merchant_id"].apply(lambda x:merchant_to_merchant_group_id[x])
# df_new_merchant_trans["merchant_group_id"] = df_new_merchant_trans["merchant_id"].apply(lambda x:merchant_to_merchant_group_id[x])

In [15]:
# from dask import dataframe as dd 
# from distributed import Client, LocalCluster
# cluster = LocalCluster()

# df_hist_trans_dd = dd.from_pandas(df_hist_trans, npartitions=100)

# df_new_merchant_trans_dd = dd.from_pandas(df_new_merchant_trans, npartitions=100)

# merchant_aggregates_dd = {}
# for col in ["merchant_group_id","merchant_category_id","subsector_id"]:
#     merchant_aggregates_dd[col] = dd.from_pandas(merchant_aggregates[col], npartitions=100)

In [16]:
# df_hist_trans.to_csv("df_hist_trans.csv")
# .to_csv("df_hist_trans.csv")

In [17]:
# import gc
# for col in ["merchant_group_id","merchant_category_id","subsector_id"]:
#     print("doing join for " , col)
#     df_hist_trans_dd = df_hist_trans_dd.set_index(col)
#     df_hist_trans_dd = df_hist_trans_dd.merge(merchant_aggregates_dd[col],on=col,how='left')
    
#     df_hist_trans_dd = df_hist_trans_dd.reset_index()

# df_hist_trans_dd.compute()
    
# df_hist_trans_dd.to_csv("/df_hist_trans/df_hist_trans.csv")

# del df_hist_trans
# gc.collect()

In [18]:
# for col in ["merchant_group_id","merchant_category_id","subsector_id"]:
#     print("doing join for " , col)
#     df_new_merchant_trans.set_index(col,inplace=True)
#     df_new_merchant_trans = df_new_merchant_trans.merge(merchant_aggregates[col],on=col,how='left')
    
#     df_new_merchant_trans.reset_index(inplace=True)
    
# df_new_merchant_trans.to_csv("df_new_merchant_trans.csv")

# del df_new_merchant_trans
# gc.collect()

In [19]:
# df_hist_trans.to_csv("df_hist_trans.csv")

In [20]:
# df_new_merchant_trans.to_csv("df_new_merchant_trans.csv")

In [21]:
holidays = np.asarray([np.datetime64(date) for date in ['2017-01-01','2017-02-27','2017-02-28','2017-03-01','2017-04-14','2017-04-16','2017-04-21','2017-05-01','2017-06-15','2017-09-07','2017-10-12','2017-11-02','2017-11-15','2017-12-25','2018-01-01','2018-02-12','2018-02-13','2018-03-30','2018-04-21','2018-05-01','2018-05-31','2018-09-07','2018-10-12','2018-11-02','2018-11-15','2018-11-20','2018-12-25']])
seasons = [0,0,0,1,1,1,2,2,2,4,4,4]

def closest_holiday_days(x):
    deltas = holidays - np.datetime64(x)
    vect_abs = np.vectorize(abs)
    vect_days = np.vectorize(lambda x: x)
    absed = vect_abs(vect_days(deltas))
    return min(absed).item().days

In [22]:
def is_in_business_hours(x):
    if (x >= 9) and (x <= 20 ):
        return 1
    else:
        return 0   

def is_close_to_pay_day(x):
    if (x > 27) or (x < 4):
        return 1
    else:
        return 0

from math import ceil

def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))    

In [23]:
is_close_to_pay_day(28)

1

In [24]:
%timeit closest_holiday_days(datetime.datetime.today())

235 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
import dask.dataframe as dd

# df_hist_trans = df_hist_trans.head(10000)
# df_new_merchant_trans = df_new_merchant_trans.head(10000)

tqdm.pandas()


for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])    
    print("processsed purchase_date")
    df['year'] = df['purchase_date'].dt.year
    print("year") 
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    print("wof")     
    df['month'] = df['purchase_date'].dt.month
    print("month")         
    df['season'] = df['month'].apply(lambda x:seasons[x-1])
    print("season")    
    df['closest_date_to_holiday'] = df['purchase_date'].progress_apply(closest_holiday_days)
    print("holiday")             
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    print("dayofweek")    
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    print("weekend")        
    df['hour'] = df['purchase_date'].dt.hour
    print("hour")    
    df['weekofmonth'] = df['purchase_date'].progress_apply(week_of_month)
    print('weekofmonth')    
#     df['is_in_business_hours'] = df['hour'].progress_apply(is_in_business_hours)
#     print('is_in_business_hours')
#     df['is_close_to_pay_day'] = df['month'].progress_apply(is_close_to_pay_day)
#     print('is_close_to_pay_day')    
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    print("AF")            
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    print("cat_1")    
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    print("month_diff")                
    df['month_diff'] += df['month_lag']

processsed purchase_date
year
wof
month


  0%|          | 0/29112361 [00:00<?, ?it/s]

season


 56%|█████▌    | 16207485/29112361 [1:36:31<1:39:31, 2161.00it/s]  

In [ ]:
pd.set_option('display.width', -1)

In [1]:
df_hist_trans.head(5)

NameError: name 'df_hist_trans' is not defined

In [ ]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [ ]:
def write_df_hist():
    df_hist_trans.to_csv("df_hist_trans.csv")

def new_merchant_trans():
    df_new_merchant_trans.to_csv("df_new_merchant_trans.csv")

In [ ]:
from multiprocessing import Process
p = Process(target=write_df_hist)
p.start()

# df_hist_trans.to_csv("df_hist_trans.csv")
# df_new_merchant_trans.to_csv("df_new_merchant_trans.csv")

In [ ]:
from multiprocessing import Process
p = Process(target=new_merchant_trans)
p.start()

In [ ]:
df_hist_trans = pd.read_csv("df_hist_trans.csv")

df_new_merchant_trans = pd.read_csv("df_new_merchant_trans.csv")

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')


In [9]:
del df_hist_trans["Unnamed: 0"]

In [ ]:
pd.set_option('display.width', -1)

In [ ]:
df_hist_trans['purchase_date'] = pd.to_datetime(df_hist_trans['purchase_date'])    
df_new_merchant_trans['purchase_date'] = pd.to_datetime(df_new_merchant_trans['purchase_date'])    

In [ ]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']

aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['season'] = ['sum','max','min','mean','var']
# aggs['is_in_business_hours'] = ['sum','max','min','mean','var']
# aggs['is_close_to_pay_day'] = ['sum','max','min','mean','var']
aggs['weekofmonth'] = ['sum','max','min','mean','var']
aggs['closest_date_to_holiday'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df_hist_trans[col+'_mean'] = df_hist_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns('hist',aggs)
df_hist_trans_group = df_hist_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

In [ ]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['season'] = ['sum','max','min','mean','var']
# aggs['is_in_business_hours'] = ['sum','max','min','mean','var']
# aggs['is_close_to_pay_day'] = ['sum','max','min','mean','var']
aggs['weekofmonth'] = ['sum','max','min','mean','var']
aggs['closest_date_to_holiday'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']
    
new_columns = get_new_columns('new_hist',aggs)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['new_hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

In [ ]:
del df_hist_trans;gc.collect()
del df_new_merchant_trans;gc.collect()
df_train.head(5)


In [ ]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

In [ ]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']

for f in ['feature_1','feature_2','feature_3']:
    order_label = df_train.groupby([f])['outliers'].mean()
    df_train[f] = df_train[f].map(order_label)
    df_test[f] = df_test[f].map(order_label)


In [ ]:
def write_df_train():
    df_train.to_csv("elo_train.csv")

from multiprocessing import Process
p = Process(target=write_df_train)
p.start()

In [ ]:
def write_df_test():
    df_test.to_csv("elo_test.csv")

from multiprocessing import Process
p = Process(target=write_df_test)
p.start()

In [ ]:
df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers']]
target = df_train['target']
del df_train['target']

In [ ]:
df_train.head()

In [ ]:
df_train.isna().sum()

In [138]:
df_train = pd.read_csv("elo_train.csv")
df_test = pd.read_csv("elo_test.csv")

In [139]:
x = df_train.columns.to_series().groupby(df.dtypes).groups
x

{dtype('<M8[ns]'): Index(['first_active_month'], dtype='object'),
 dtype('int64'): Index(['hist_month_nunique', 'hist_hour_nunique', 'hist_weekofyear_nunique',
        'hist_dayofweek_nunique', 'hist_year_nunique',
        'hist_subsector_id_nunique', 'hist_merchant_id_nunique',
        'hist_merchant_category_id_nunique', 'hist_installments_sum',
        'hist_installments_max', 'hist_installments_min', 'hist_season_sum',
        'hist_season_max', 'hist_season_min',
        'hist_closest_date_to_holiday_sum', 'hist_closest_date_to_holiday_max',
        'hist_closest_date_to_holiday_min', 'hist_month_lag_max',
        'hist_month_lag_min', 'hist_authorized_flag_sum', 'hist_weekend_sum',
        'hist_category_1_sum', 'hist_card_id_size', 'hist_purchase_date_diff',
        'hist_purchase_date_uptonow'],
       dtype='object'),
 dtype('float64'): Index(['feature_1', 'feature_2', 'feature_3', 'hist_purchase_amount_sum',
        'hist_purchase_amount_max', 'hist_purchase_amount_min',
    

In [140]:
col_seris = df_train.isna().sum() == 0
col_df = pd.DataFrame({'email':col_seris.index, 'has_na':col_seris.values})
na_columns_train = col_df[col_df["has_na"] == False].email.tolist()

col_seris = df_test.isna().sum() == 0
col_df = pd.DataFrame({'email':col_seris.index, 'has_na':col_seris.values})
na_columns_test = col_df[col_df["has_na"] == False].email.tolist()

In [141]:
na_columns = set(na_columns_train).union(set(na_columns_test))

In [142]:
print(na_columns)
print(len(na_columns))

{'new_hist_category_3_mean_mean', 'month', 'card_id_total', 'new_hist_category_1_mean', 'first_active_month', 'new_hist_month_lag_min', 'new_hist_year_nunique', 'new_hist_merchant_id_nunique', 'new_hist_month_diff_mean', 'new_hist_purchase_amount_var', 'new_hist_month_lag_max', 'new_hist_season_min', 'weekofyear', 'new_hist_card_id_size', 'new_hist_purchase_amount_max', 'new_hist_purchase_amount_min', 'new_hist_closest_date_to_holiday_sum', 'new_hist_first_buy', 'dayofweek', 'new_hist_closest_date_to_holiday_min', 'new_hist_season_var', 'new_hist_category_1_sum', 'new_hist_weekend_sum', 'new_hist_purchase_date_average', 'new_hist_closest_date_to_holiday_max', 'new_hist_installments_sum', 'new_hist_month_nunique', 'new_hist_category_2_mean_mean', 'new_hist_weekend_mean', 'new_hist_installments_mean', 'new_hist_month_lag_var', 'new_hist_purchase_amount_mean', 'new_hist_subsector_id_nunique', 'new_hist_closest_date_to_holiday_var', 'new_hist_installments_var', 'new_hist_month_lag_mean', '

In [146]:
# from sklearn.preprocessing import PolynomialFeatures
# # df_train["card_id"] = df_train_string["card_id"]
# df_train_string = df_train[["card_id","first_active_month"]]

# del df_train["card_id"]
# del df_train["first_active_month"]

# df_train_new = PolynomialFeatures(degree=2).fit(df_train)
# # df_test = PolynomialFeatures(degree=2).fit(df_test)

In [ ]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.005,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = df_train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration) / folds.n_splits

np.sqrt(mean_squared_error(oof, target))

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="Feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')


In [ ]:
sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("submission_week_of_month.csv", index=False)

In [11]:
pd.set_option('display.max_columns', None)

In [178]:
from scipy import stats

In [231]:
df_train.columns.tolist()

['Unnamed: 0',
 'first_active_month',
 'card_id',
 'feature_1',
 'feature_2',
 'feature_3',
 'hist_month_nunique',
 'hist_hour_nunique',
 'hist_weekofyear_nunique',
 'hist_dayofweek_nunique',
 'hist_year_nunique',
 'hist_subsector_id_nunique',
 'hist_merchant_id_nunique',
 'hist_merchant_category_id_nunique',
 'hist_purchase_amount_sum',
 'hist_purchase_amount_max',
 'hist_purchase_amount_min',
 'hist_purchase_amount_mean',
 'hist_purchase_amount_var',
 'hist_installments_sum',
 'hist_installments_max',
 'hist_installments_min',
 'hist_installments_mean',
 'hist_installments_var',
 'hist_season_sum',
 'hist_season_max',
 'hist_season_min',
 'hist_season_mean',
 'hist_season_var',
 'hist_closest_date_to_holiday_sum',
 'hist_closest_date_to_holiday_max',
 'hist_closest_date_to_holiday_min',
 'hist_closest_date_to_holiday_mean',
 'hist_closest_date_to_holiday_var',
 'hist_purchase_date_max',
 'hist_purchase_date_min',
 'hist_month_lag_max',
 'hist_month_lag_min',
 'hist_month_lag_mean',
 

In [226]:
len(a)

60

In [171]:
mode([1,3,2,3,5])[0][0]

3

In [ ]:
# holidays = [1,3,2,3,5]
print(holidays)
a = [date for date in holidays]

In [ ]:
min(df_new_merchant_trans["purchase_date"])

In [ ]:
max(df_new_merchant_trans["purchase_date"])

In [ ]:
df_new_merchant_trans.dtypes

In [ ]:
[pd.to_datetime(date) for date in ["Jan 01, 2017","Feb 27, 2017","Feb 28, 2017","Mar 01, 2017","Apr 14, 2017","Apr 16, 2017","Apr 21, 2017","May 01, 2017","Jun 15, 2017","Sep 07, 2017","Oct 12, 2017","Nov 02, 2017","Nov 15, 2017","Dec 25, 2017","1 Jan 2018","12 Feb 2018","13 Feb 2018","30 Mar 2018","21 Apr 2018","1 May 2018","31 May 2018","7 Sep 2018","12 Oct 2018","2 Nov 2018","15 Nov 2018","20 Nov 2018","25 Dec 2018"]]

In [ ]:
(pd.to_datetime("Apr 21, 2017") - pd.to_datetime('2017-01-01 00:00:08')).days

In [29]:
holidays = np.asarray([pd.to_datetime(date) for date in ["Jan 01, 2017","Feb 27, 2017","Feb 28, 2017","Mar 01, 2017","Apr 14, 2017","Apr 16, 2017","Apr 21, 2017","May 01, 2017","Jun 15, 2017","Sep 07, 2017","Oct 12, 2017","Nov 02, 2017","Nov 15, 2017","Dec 25, 2017","1 Jan 2018","12 Feb 2018","13 Feb 2018","30 Mar 2018","21 Apr 2018","1 May 2018","31 May 2018","7 Sep 2018","12 Oct 2018","2 Nov 2018","15 Nov 2018","20 Nov 2018","25 Dec 2018"]])

In [34]:
[i.strftime("%Y-%m-%d") for i in holidays]

['2017-01-01',
 '2017-02-27',
 '2017-02-28',
 '2017-03-01',
 '2017-04-14',
 '2017-04-16',
 '2017-04-21',
 '2017-05-01',
 '2017-06-15',
 '2017-09-07',
 '2017-10-12',
 '2017-11-02',
 '2017-11-15',
 '2017-12-25',
 '2018-01-01',
 '2018-02-12',
 '2018-02-13',
 '2018-03-30',
 '2018-04-21',
 '2018-05-01',
 '2018-05-31',
 '2018-09-07',
 '2018-10-12',
 '2018-11-02',
 '2018-11-15',
 '2018-11-20',
 '2018-12-25']